In [8]:
%matplotlib inline
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import importlib
import sys

import numpy as np
import pandas as pd

import util

cur_dir = os.path.abspath('.')
root_dir = os.path.dirname(cur_dir)
sys.path.append(os.path.join(cur_dir, '3DUnetCNN'))

import unet3d.my_generator
from unet3d.training import load_old_model

In [9]:
model = load_old_model('./model_unet3d_20190722_122528.h5')

Loading pre-trained model


In [10]:
df_gt = pd.read_csv(os.path.join(cur_dir, 'annotation_voxel.csv'))
df_gt.head(10)

,seriesuid,coordX,coordY,coordZ,diameterX,diameterY,diameterZ,label
0,636990,214,257,28,31,21,3,32
1,636990,210,232,31,15,20,3,32
2,636990,237,258,35,27,28,6,32
3,636990,158,218,33,13,12,1,1
4,636990,171,200,34,11,15,1,1
5,636990,270,252,36,18,10,1,31
6,696540,172,365,17,53,54,8,5
7,696540,310,282,19,25,18,1,5
8,696540,164,388,24,26,9,1,5
9,696540,126,367,33,16,16,1,5


In [11]:
importlib.reload(unet3d.my_generator)
importlib.reload(util)

res = []
uid_list = df_gt['seriesuid'].unique()[:5]
predictions = []
for seriesuid in uid_list:
#     seriesuid = '636990'
    data = np.load('./data/3d/{}_data.npy'.format(seriesuid))
    prediction = unet3d.my_generator.predict_sample(model, data)
    predictions.append(prediction)
    targets = util.parse_prediction_3d(prediction, score_th=0.02, side_len_th=3)
    res += [[seriesuid] + row for row in targets]
    
res_df = pd.DataFrame(res, columns=['seriesuid', 'coordX', 'coordY', 'coordZ', 'class', 'probability'])

In [12]:
res_df

,seriesuid,coordX,coordY,coordZ,class,probability
0,636990,183,184,107,5,4.689107
1,696540,194,224,159,5,5.374519
2,510918,204,168,192,5,8.787139
3,679846,162,164,144,5,19.956238
4,408822,204,197,105,5,16.540879


In [14]:
util.plot_ct_as_slide(predictions[1][1], interval=50)